In [1]:
# 9.13 使用元类控制实例的创建
"""问题
你想通过改变实例创建方式来实现单例、缓存或其他类似的特性。

解决方案
Python程序员都知道，如果你定义了一个类，就能像函数一样的调用它来创建实例，例如："""
class Spam:
    def __init__(self, name):
        self.name = name

a = Spam('Guido')
b = Spam('Diana')

In [2]:
"""如果你想自定义这个步骤，你可以定义一个元类并自己实现 __call__() 方法。

为了演示，假设你不想任何人创建这个类的实例："""
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can not instantiate directly")
        
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [3]:
"""这样的话，用户只能调用这个类的静态方法，而不能使用通常的方法来创建它的实例。例如："""
Spam.grok(42)

Spam.grok


In [4]:
s = Spam()

TypeError: Can not instantiate directly

In [5]:
"""现在，假如你想实现单例模式（只能创建唯一实例的类），实现起来也很简单："""
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self._instance = None
        super().__init__(*args, **kwargs)
        
    def __call__(self, *args, **kwargs):
        if self._instance is None:
            self._instance = super().__call__(*args, **kwargs)
            return self._instance
        else:
            return self._instance
        
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

In [6]:
"""那么Spam类就只能创建唯一的实例了，演示如下："""
a = Spam()


Creating Spam


In [7]:
b = Spam()

In [8]:
a is b

True

In [15]:
"""最后，假设你想创建8.25小节中那样的缓存实例。下面我们可以通过元类来实现："""
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()

    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj

# Example
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name

In [16]:
"""然后我也来测试一下："""
a = Spam('Guido')


Creating Spam('Guido')


In [17]:
b = Spam('Diana')

Creating Spam('Diana')


In [18]:
c = Spam('Guido')

In [19]:
a is b

False

In [20]:
a is c 

True

In [21]:
"""讨论
利用元类实现多种实例创建模式通常要比不使用元类的方式优雅得多。

假设你不使用元类，你可能需要将类隐藏在某些工厂函数后面。 比如为了实现一个单例，你你可能会像下面这样写："""
class _Spam:
    def __init__(self):
        print('Creating Spam')

_spam_instance = None

def Spam():
    global _spam_instance

    if _spam_instance is not None:
        return _spam_instance
    else:
        _spam_instance = _Spam()
        return _spam_instance